In [114]:
import pandas as pd
import numpy as np
import random
import os
import networkx as nx
import tqdm

import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from pylab import rcParams
from multiprocessing import Pool, cpu_count
from functools import partial


from importlib import reload
import merf.merf
rcParams['figure.figsize'] = 18, 15

%matplotlib inline

In [115]:
ideologies_dir = "./"
followers_dir = "../followers/"
geolocations_dir = "./"

In [116]:
ideals1 = pd.read_csv(ideologies_dir + "user-ideal-points-201807-000000000000.csv", usecols=["id_str", "theta"])
ideals2 = pd.read_csv(ideologies_dir + "user-ideal-points-201807-000000000001.csv",  usecols=["id_str", "theta"])
ideals3 =  pd.read_csv(ideologies_dir + "user-ideal-points-201807-000000000002.csv",  usecols=["id_str", "theta"])

ideals = ideals1.append(ideals2).append(ideals3)

In [117]:
user_ids = os.listdir(followers_dir)

In [118]:
geos1 = pd.read_csv(geolocations_dir + "already_located_random_users_machine00.csv", usecols=["user_id", "longitude", "latitude", "raw_location", "country"])
geos2 = pd.read_csv(geolocations_dir + "already_located_random_users_machine01.csv", usecols=["user_id", "longitude", "latitude", "raw_location", "country"])
geos3 = pd.read_csv(geolocations_dir + "already_located_random_users_machine02.csv", usecols=["user_id", "longitude", "latitude", "raw_location", "country"])

geos = geos1.append(geos2).append(geos3)

In [ ]:
geos

In [119]:
sample_size = 2000
ideal_ids = ideals["id_str"]
geo_ids = geos["user_id"]

In [120]:
combined_ids = np.intersect1d(np.intersect1d(ideal_ids.values, geo_ids.values), user_ids)
combined_ids = combined_ids.tolist()

In [122]:
random_users = combined_ids

In [123]:
fname = lambda user_id: followers_dir + str(user_id) + "/" + os.listdir(followers_dir + str(user_id))[0]
result = []
graph = {}
iterations = 0
to_visit = []

In [124]:
for u in tqdm.tqdm(combined_ids):
    df = pd.read_csv(fname(u))
    graph[u] = df[df["user_id_followers"].isin(combined_ids)]

100%|██████████| 2762/2762 [00:07<00:00, 382.38it/s]


In [125]:
graph

{'100132523': Empty DataFrame
 Columns: [user_id_followers]
 Index: [], '1001917210099580928':       user_id_followers
 333           274218971
 1050         2365679371
 1237         3950218941
 1344          751970174
 1632          201931078
 1646          556643174
 2101           58903969
 2532         1365557564
 3386          138225977, '100216363': Empty DataFrame
 Columns: [user_id_followers]
 Index: [], '1003475954': Empty DataFrame
 Columns: [user_id_followers]
 Index: [], '100386596': Empty DataFrame
 Columns: [user_id_followers]
 Index: [], '100511810': Empty DataFrame
 Columns: [user_id_followers]
 Index: [], '100513911': Empty DataFrame
 Columns: [user_id_followers]
 Index: [], '100638521':      user_id_followers
 37           257467770
 261         2301628435, '100743872': Empty DataFrame
 Columns: [user_id_followers]
 Index: [], '100782738': Empty DataFrame
 Columns: [user_id_followers]
 Index: [], '1008060669005107200': Empty DataFrame
 Columns: [user_id_followers]
 In

Backward propagating...


,node_1,node_2
0,1001917210099580928,274218971
1,1001917210099580928,2365679371
2,1001917210099580928,3950218941
3,1001917210099580928,751970174
4,1001917210099580928,201931078
...,...,...
1,967007887,2365679371
0,97044552,261918703
0,972551436,73607865
0,989623975,527084853


In [121]:
fname = lambda user_id: followers_dir + str(user_id) + "/" + os.listdir(followers_dir + str(user_id))[0]
result = {}
with tqdm.tqdm_notebook(total = sample_size, desc = "sample iterations") as t:
    while len(result.keys()) < sample_size:
        random_users = random.sample(combined_ids, sample_size)
        collected = 0
        for u in tqdm.tqdm_notebook(random_users, desc = "collecting random uers", leave = False):
            if u not in result:
                df = pd.read_csv(fname(u))
                subgraph = df[df["user_id_followers"].isin(random_users)]
                if len(subgraph) > 0:
                    result[u] = subgraph
                    collected += 1
            
        t.update(collected)
        collected = 0
for u in tqdm.tqdm_notebook(result.keys()):
    df = pd.read_csv(fname(u))
    subgraph = df[df["user_id_followers"].isin(random_users)]
    result[u].append(subgraph).drop_duplicates()
    

KeyboardInterrupt: 

In [ ]:
for u in tqdm.tqdm_notebook(result.keys()):
    df = pd.read_csv(fname(u))
    subgraph = df[df["user_id_followers"].isin(random_users)]
    result[u].append(subgraph).drop_duplicates()

In [ ]:
fname = lambda user_id: followers_dir + str(user_id) + "/" + os.listdir(followers_dir + str(user_id))[0]
visited = []
graph = {}
random_users = random.sample(combined_ids, sample_size / 2)
to_visit = [random_users]
i = 0
with tqdm.tqdm_notebook(total = sample_size, desc = "sample iterations") as t:
    while len(visited) < sample_size:
        u = to_visit.pop(0)
        try:
            df = pd.read_csv(fname(u))
        except:
            continue
        visited.append(u)
        subgraph = df[df["user_id_followers"].isin(graph.keys())]
        
        to_visit += df["user_id_followers"].values.tolist()
        if len(subgraph) > 0 :
            graph[u] = subgraph
            t.update(1)
        i += 1

In [ ]:
result

In [ ]:
fname = lambda user_id: followers_dir + str(user_id) + "/" + os.listdir(followers_dir + str(user_id))[0]
result = []
graph = {}
with tqdm.tqdm_notebook(total = sample_size, desc = "sample iterations") as t:
    while len(graph.keys()) < sample_size:
        random_users = random.sample(combined_ids, sample_size)
        collected = 0
        for u in tqdm.tqdm_notebook(random_users, desc = "collecting random uers", leave = False):
            if u not in graph:
                df = pd.read_csv(fname(u))
                subgraph = df[df["user_id_followers"].isin(random_users)]
                if len(subgraph) > 0:
                    graph[u] = subgraph
                    collected += 1
            
        t.update(collected)
        collected = 0

print("Backward propagating...")
for u in tqdm.tqdm_notebook(graph.keys(), desc = "parsing collected random users"):
    df = pd.read_csv(fname(u))
    subgraph = df[df["user_id_followers"].isin(random_users)]
    subgraph = graph[u].append(subgraph).drop_duplicates()
    if mode == "LINE":
        func = lambda x: {"source_node": u, "target_node": x, "weight": 1}
    else:
        func = lambda x: {"node_1": u, "node_2": x}
    subgraph = pd.DataFrame(subgraph["user_id_followers"].apply(func).tolist())
    result.append(subgraph)

In [152]:
mode = "LINE"
print("Backward propagating...")
for u in tqdm.tqdm_notebook(graph.keys(), desc = "parsing collected random users"):
    df = pd.read_csv(fname(u))
    subgraph = df[df["user_id_followers"].isin(random_users)]
    subgraph = graph[u].append(subgraph).drop_duplicates()
    if mode == "LINE":
        func = lambda x: {"source_node": u, "target_node": x, "weight": 1}
    else:
        func = lambda x: {"node_1": u, "node_2": x}
    subgraph = pd.DataFrame(subgraph["user_id_followers"].apply(func).tolist())
    line_result.append(subgraph)

Backward propagating...


In [ ]:
fname = lambda user_id: followers_dir + str(user_id) + "/" + os.listdir(followers_dir + str(user_id))[0]
graph = {}
for u in tqdm.tqdm_notebook(combined_ids, desc = "collecting random uers", leave = False):
    df = pd.read_csv(fname(u))
    subgraph = df[df["user_id_followers"].isin(combined_ids)]
    if len(subgraph) > 0:
        graph[u] = subgraph
    else:
        combined_ids.remove(u)

result = []
print("Backward propagating...")
for u in tqdm.tqdm_notebook(graph.keys(), desc = "parsing collected random users"):
    df = pd.read_csv(fname(u))
    subgraph = df[df["user_id_followers"].isin(random_users)]
    subgraph = graph[u].append(subgraph).drop_duplicates()
    subgraph = pd.DataFrame(subgraph["user_id_followers"].apply(lambda x: {"node_1": u, "node_2": x}).tolist())
    result.append(subgraph)

In [150]:
mnf_result = result


In [153]:
line_result = pd.concat(line_result)

In [148]:
result = pd.concat(result)
result

,node_1,node_2
0,1001917210099580928,274218971
1,1001917210099580928,2365679371
2,1001917210099580928,3950218941
3,1001917210099580928,751970174
4,1001917210099580928,201931078
...,...,...
1,967007887,2365679371
0,97044552,261918703
0,972551436,73607865
0,989623975,527084853


In [ ]:
subgraph = graph[u].append(subgraph).drop_duplicates()
subgraph

In [ ]:
def g():
    filesnames = [(user_id, followers_dir + str(random_users[i]) + "/" + os.listdir(followers_dir +"/" + str(random_users[i]))[0]) for i, user_id in enumerate(random_users)]
    dfs = [pd.read_csv(f, names = [user_id], header=None) for user_id, f in filesnames]

    result = []
    for df in dfs:
        user_id = df.columns[0]
        for follower in df.values:
            result.append({
                "node_1": user_id,
                "node_2": follower
            })
%time g()

In [ ]:
def y():
    result = []
    failures = 0
    for i, user_id in enumerate(random_users):
        try:
            filename = os.listdir(followers_dir +"/" + str(random_users[i]))[0]
            followers = pd.read_csv(followers_dir + str(random_users[i]) + "/" + filename)["user_id_followers"].values

            for follower in followers:
                result.append({
                    "node_1": user_id,
                    "node_2": follower
                })
        except Exception as e:
            failures += 1

%time y()

In [ ]:
# CONSTRUCT LINE FILE
result = []
failures = 0
print("Compiling user data...")

for i, user_id in enumerate(random_users):
    try:
        followers = pd.read_csv(followers_dir + str(random_users[i]) + "/2019__10__" + str(random_users[i]) + ".csv")["user_id_followers"].values
        datum = {
            'node_id': user_id,
            'neigh_num': len(followers),
            'neighbors': followers
        }
        result.append(datum)
    except:
        failures += 1
print("writing file...")
with open("sine.txt", "w+") as f:
    f.write(str(len(result)) + " 1\n")
    for node in result:
        f.write(str(node["node_id"]) + "\n")
        f.write(str(node["neigh_num"]) + "\n")
        f.write(" ".join([str(n) for n in node["neighbors"]]) + "\n")
        f.write("1\n")
        f.write("0\n")

print("done")

In [157]:
# h = pd.DataFrame(result)
h = line_result
r = np.concatenate((h["source_node"].unique(),h["target_node"].unique()), axis=0)
rl = r.tolist()
obj = {}
for i, x in enumerate(r):
    obj[x] = i
h["source_node"] = h["source_node"].apply(lambda x: obj[x])
h["target_node"] = h["target_node"].apply(lambda x: obj[x])

# h.to_csv("followerified_with_geo.csv", index=False)

In [158]:
h.to_csv("line_result.csv", index=False)

In [ ]:
result = []
for user in h["node_1"].unique():
    followers = h[h["node_1"] == user]["node_2"].values
    datum = {
        'node_id': user,
        'neigh_num': len(followers),
        'neighbors': followers
    }
    result.append(datum)

In [ ]:
with open("sine.txt", "w+") as f:
    f.write(str(len(result)) + " 0\n")
    for node in result:
        f.write(str(node["node_id"]) + "\n")
        f.write(str(node["neigh_num"]) + "\n")
        f.write(" ".join([str(n) for n in node["neighbors"]]) + "\n")
        f.write("0\n")
        f.write("\n")

print("done")

In [159]:
line_embedding = pd.read_csv("line_embedding.csv", index_col=0)
mnf_embedding = pd.read_csv("mnf_embedding.csv")

In [162]:
vals = list(obj.values())
keys = list(obj.keys())

In [177]:
def j(x):
    mapped_id = keys[vals.index(x.name)]
    
    ideology = ideals[ideals["id_str"] == int(mapped_id)]
    geo = geos[geos["user_id"] == int(mapped_id)]
    
    x["mapped_id"] = mapped_id
    x['theta'] = ideology["theta"].iloc[0]
    x["longitude"] = geo["longitude"].iloc[0]
    x["latitude"] = geo["latitude"].iloc[0]
    x["raw_location"] = geo["raw_location"].iloc[0]
    x["country"] = geo["country"].iloc[0]
    return x

# dataset = embedding.apply(j, axis=1)
line_dataset = line_embedding.apply(j, axis=1)
mnf_dataset = mnf_embedding.apply(j, axis=1)

In [179]:
X_line = line_dataset[line_dataset.columns.difference(['mapped_id', 'theta', "country","raw_location", "latitude", "longitude"])]
y_line = line_dataset[["theta"]]
y_line_multi = line_dataset[["theta", "latitude", "longitude"]]

In [212]:
X_mnf = mnf_dataset[mnf_dataset.columns.difference(['mapped_id', 'theta', "country","raw_location", "latitude", "longitude"])]
y_mnf = mnf_dataset[["theta"]]
y_mnf_multi = mnf_dataset[["theta", "latitude", "longitude"]]

In [213]:
X_mnf_train, X_mnf_test, y_mnf_train, y_mnf_test = train_test_split(X_mnf, y_mnf, test_size=0.33, random_state=42)
y_mnf_train = y_mnf_train.fillna(0)
y_mnf_test = y_mnf_test.fillna(0)

In [207]:
X_line_train, X_line_test, y_line_train, y_line_test = train_test_split(X_line, y_line, test_size=0.33, random_state=42)
y_line_train = y_line_train.fillna(0)
y_line_test = y_line_test.fillna(0)

## M-NMF Ideaology Prediction Score using Linear Regression

|             | Ideaology Prediction | Ideology + Geolocation
| ----------- | ----------- | -----
| MNMF  | -0.0.37365315089042994
       |
| LINE  | Text        |

In [214]:
model = LinearRegression()
model.fit(X_mnf_train, y_mnf_train)
model.score(X_mnf_test, y_mnf_test)

-0.2655526498888303

## LINE Ideaology Prediction Score using Linear Regression

In [210]:
model = LinearRegression()
model.fit(X_line_train, y_line_train)
model.score(X_line_test, y_line_test)

-0.37365315089042994

In [215]:
X_mnf_train, X_mnf_test, y_mnf_train, y_mnf_test = train_test_split(X_mnf, y_mnf_multi, test_size=0.33, random_state=42)
y_mnf_train = y_mnf_train.fillna(0)
y_mnf_test = y_mnf_test.fillna(0)

X_line_train, X_line_test, y_line_train, y_line_test = train_test_split(X_line, y_line_multi, test_size=0.33, random_state=42)
y_line_train = y_line_train.fillna(0)
y_line_test = y_line_test.fillna(0)

## M-NMF Ideaology & Geolocation Prediction Score using Random Forest Regression

In [216]:
tree_model = RandomForestRegressor()
tree_model.fit(X_mnf_train, y_mnf_train)
tree_model.score(X_mnf_test, y_mnf_test)

/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


-0.31522200277874185

## LINE Ideaology & Geolocation Prediction Score using Random Forest Regression

In [217]:
tree_model = RandomForestRegressor()
tree_model.fit(X_line_train, y_line_train)
tree_model.score(X_line_test, y_line_test)

/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


-0.26061137560660297